# Guía 2: Tokens y Embeddings

## Descripción

En esta guía vas a explorar dos conceptos fundamentales en el procesamiento del lenguaje natural: **tokenización** y **embeddings**. Estos son los pilares sobre los que funcionan todos los modelos de lenguaje modernos.

Al finalizar esta guía vas a poder:
- Comprender qué son los tokens y cómo diferentes modelos tokenizan el texto
- Visualizar y comparar estrategias de tokenización entre modelos
- Entender qué son los embeddings y cómo representan significado
- Obtener embeddings contextualizados y embeddings de oraciones completas
- Aplicar embeddings para tareas de búsqueda semántica y similitud textual

---

## Configuración del Entorno

### Requisitos

Para ejecutar este notebook necesitás:

1. **Google Colab con GPU activada:**
   - Andá a `Entorno de ejecución > Cambiar tipo de entorno de ejecución`
   - Seleccioná `Acelerador de hardware > GPU`
   - Elegí `Tipo de GPU > T4`

2. **Conexión a Internet** para descargar modelos y bibliotecas

Nota: Algunos ejemplos funcionan en CPU, pero para cargar modelos grandes es recomendable usar GPU.

## Instalación de Dependencias

Instalamos las bibliotecas necesarias para trabajar con tokens y embeddings.

In [1]:
%%capture
# Actualizamos pip y limpiamos instalaciones previas
!pip install --upgrade pip
!pip uninstall -y transformers accelerate

# Instalamos las bibliotecas necesarias
!pip install transformers accelerate
!pip install sentence-transformers gensim scikit-learn numpy scipy

### Bibliotecas Instaladas

- **transformers**: Para trabajar con modelos de lenguaje y tokenizadores
- **accelerate**: Acelera la inferencia de modelos
- **sentence-transformers**: Modelos especializados en embeddings de oraciones
- **gensim**: Biblioteca para modelado de tópicos y similitud semántica
- **scikit-learn**: Herramientas de machine learning (cálculo de similitudes)
- **numpy, scipy**: Operaciones numéricas y científicas

## Marco Teórico

### ¿Qué es la Tokenización?

Los modelos de lenguaje no procesan texto directamente, sino que trabajan con **tokens**. La **tokenización** es el proceso de dividir el texto en unidades más pequeñas que el modelo puede procesar.

Un token puede ser:
- Una palabra completa: `"casa"` → `["casa"]`
- Una subpalabra: `"jugando"` → `["jug", "ando"]`
- Un carácter individual: `"a"` → `["a"]`
- Puntuación o espacios especiales

### ¿Por qué Subpalabras?

Los tokenizadores modernos usan algoritmos como **BPE (Byte-Pair Encoding)** o **WordPiece** que dividen palabras en subpalabras. Ventajas:

1. **Vocabulario finito**: En lugar de millones de palabras, usan 30,000-50,000 subpalabras
2. **Manejo de palabras raras**: Pueden representar palabras que nunca vieron durante el entrenamiento
3. **Eficiencia morfológica**: Capturan raíces y afijos ("jugando" = "jug" + "ando")
4. **Multilingüismo**: Un mismo tokenizador puede manejar múltiples idiomas

### ¿Qué son los Embeddings?

Los **embeddings** son representaciones numéricas (vectores) de palabras, oraciones o documentos. Cada token se convierte en un vector de números (típicamente 256, 512, 768 o 1024 dimensiones).

Propiedades clave:
- Palabras similares tienen vectores similares
- Capturan relaciones semánticas: `rey - hombre + mujer ≈ reina`
- Permiten operaciones matemáticas con significado

### Tipos de Embeddings

1. **Embeddings estáticos** (Word2Vec, GloVe): Cada palabra tiene un único vector fijo
   - Problema: "banco" (institución) y "banco" (asiento) tienen el mismo vector

2. **Embeddings contextualizados** (BERT, GPT): El vector cambia según el contexto
   - "banco" tiene vectores diferentes en "banco de datos" vs "sentarse en el banco"

3. **Embeddings de oraciones** (Sentence-BERT): Representan oraciones completas en un solo vector
   - Útiles para búsqueda semántica, clustering, clasificación

## Parte 1: Explorando la Tokenización

Vamos a comparar cómo diferentes modelos tokenizan el mismo texto.

In [2]:
from transformers import AutoTokenizer

# Función auxiliar para visualizar tokens con colores
colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def mostrar_tokens(texto, nombre_tokenizador):
    """
    Visualiza cómo un tokenizador divide el texto.

    Parámetros:
    - texto: String a tokenizar
    - nombre_tokenizador: Identificador del modelo en Hugging Face
    """
    print(f"\n{'='*60}")
    print(f"Tokenizador: {nombre_tokenizador}")
    print(f"{'='*60}")

    tokenizer = AutoTokenizer.from_pretrained(nombre_tokenizador)
    token_ids = tokenizer(texto).input_ids

    # Mostramos cada token con un color diferente
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )
    print(f"\n\nTotal de tokens: {len(token_ids)}")

### Texto de Prueba

Preparamos un texto que incluye diferentes desafíos para los tokenizadores:
- Texto en español y mayúsculas
- Emoji y caracteres especiales (🧉 ñ)
- Código Python
- Espacios y tabulaciones
- Operaciones matemáticas
- Expresiones coloquiales argentinas

In [3]:
# Texto de ejemplo con varios casos interesantes
texto = """
Español y MAYÚSCULAS
🧉 ñ
show_tokens False None elif == >= else: dos tabs:"    " Tres tabs: "       "
12.0*50=600
El mate está muy caliente
"""

print("Texto original:")
print(texto)

Texto original:

Español y MAYÚSCULAS
🧉 ñ
show_tokens False None elif == >= else: dos tabs:"    " Tres tabs: "       "
12.0*50=600
El mate está muy caliente



### Comparación de Tokenizadores

Vamos a probar cuatro tokenizadores diferentes:

1. **BERT uncased**: Convierte todo a minúsculas, no distingue mayúsculas
2. **BERT cased**: Mantiene mayúsculas y minúsculas
3. **Phi-3**: Tokenizador moderno optimizado para modelos instruction-tuned
4. **GPT-2**: Tokenizador clásico basado en BPE

In [4]:
# BERT uncased (sin distinción de mayúsculas)
mostrar_tokens(texto, "bert-base-uncased")


Tokenizador: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[CLS] es ##pan ##ol y may ##us ##cula ##s [UNK] n show _ token ##s false none eli ##f = = > = else : dos tab ##s : " " tres tab ##s : " " 12 . 0 * 50 = 600 el mate est ##a mu ##y cal ##iente [SEP] 

Total de tokens: 53


In [5]:
# BERT cased (con distinción de mayúsculas)
mostrar_tokens(texto, "bert-base-cased")


Tokenizador: bert-base-cased


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[CLS] E ##sp ##a ##ño ##l y MA ##Y ##Ú ##SC ##U ##LA ##S [UNK] ñ show _ token ##s F ##als ##e None el ##if = = > = else : dos ta ##bs : " " T ##res ta ##bs : " " 12 . 0 * 50 = 600 El mate est ##á m ##uy ca ##lient ##e [SEP] 

Total de tokens: 62


In [6]:
# Phi-3 (modelo moderno)
mostrar_tokens(texto, "microsoft/Phi-3-mini-4k-instruct")


Tokenizador: microsoft/Phi-3-mini-4k-instruct


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

 
 Esp añ ol y MA Y Ú SC UL AS 
 � � � �  ñ 
 show _ to kens False None elif == >= else : dos tabs :"    " T res tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 El mate está muy cal iente 
 

Total de tokens: 63


In [7]:
# GPT-2 (tokenizador clásico)
mostrar_tokens(texto, "gpt2")


Tokenizador: gpt2


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 E sp a ñ ol  y  MAY � � SC UL AS 
 � � �  � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  dos  tabs :"        "  T res  tabs :  "              " 
 12 . 0 * 50 = 600 
 El  mate  est á  m uy  cal ient e 
 

Total de tokens: 71


### Observaciones Clave

Al comparar los tokenizadores, notá:

1. **Manejo de espacios**: Algunos añaden espacios como parte del token, otros no
2. **Caracteres especiales**: Diferentes estrategias para emojis (🧉) y acentos (ñ)
3. **Números y operadores**: Algunos los agrupan, otros los separan
4. **Eficiencia**: Menos tokens = procesamiento más eficiente
5. **Español**: Modelos entrenados principalmente en inglés fragmentan más las palabras españolas

## Parte 2: Embeddings Contextualizados

Ahora vamos a obtener embeddings contextualizados usando un modelo BERT.

In [8]:
from transformers import AutoModel, AutoTokenizer

# Cargamos un modelo BERT pequeño
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

print("Modelo DeBERTa cargado exitosamente")

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

Modelo DeBERTa cargado exitosamente


### DeBERTa: Modelo de Embeddings Contextualizados

**DeBERTa** (Decoding-enhanced BERT with disentangled attention) es una mejora de BERT que:
- Genera embeddings que consideran el contexto completo de la oración
- Usa atención mejorada para capturar relaciones entre palabras
- Es más eficiente que BERT tradicional

Usamos la versión `v3-xsmall` porque es:
- **Liviana**: Solo 22M parámetros
- **Rápida**: Inferencia veloz incluso en CPU
- **Suficiente**: Adecuada para demostración educativa

In [9]:
# Procesamos una oración simple
oracion = 'Hola mundo'

# Tokenizamos
# return_tensors='pt': Devuelve tensores de PyTorch
tokens = tokenizer(oracion, return_tensors='pt')

print("Tokens generados:")
print(tokens)
print("\nIDs de tokens:")
print(tokens['input_ids'])
print("\nDecodificación de cada token:")
for token_id in tokens['input_ids'][0]:
    print(f"ID {token_id}: '{tokenizer.decode(token_id)}'")

Tokens generados:
{'input_ids': tensor([[   1,  725, 3019, 9500,  139,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

IDs de tokens:
tensor([[   1,  725, 3019, 9500,  139,    2]])

Decodificación de cada token:
ID 1: '[CLS]'
ID 725: 'H'
ID 3019: 'ola'
ID 9500: ' mund'
ID 139: 'o'
ID 2: '[SEP]'


In [10]:
# Obtenemos los embeddings
# **tokens: Desempaqueta el diccionario como argumentos nombrados
# [0]: Tomamos la primera salida (hidden states)
output = model(**tokens)[0]

print("Forma del tensor de embeddings:")
print(output.shape)
print("\nInterpretación:")
print(f"- Batch size: {output.shape[0]} (cantidad de oraciones procesadas simultáneamente)")
print(f"- Sequence length: {output.shape[1]} (cantidad de tokens en la oración)")
print(f"- Hidden size: {output.shape[2]} (dimensiones del embedding de cada token)")

Forma del tensor de embeddings:
torch.Size([1, 6, 384])

Interpretación:
- Batch size: 1 (cantidad de oraciones procesadas simultáneamente)
- Sequence length: 6 (cantidad de tokens en la oración)
- Hidden size: 384 (dimensiones del embedding de cada token)


### Interpretando la Salida

El tensor resultante tiene forma `[1, n_tokens, 384]`:

- **1**: Un solo ejemplo (batch size = 1)
- **n_tokens**: Número de tokens en la oración (incluyendo tokens especiales [CLS], [SEP])
- **384**: Dimensiones del embedding (cada token se representa con 384 números)

Cada token ahora es un vector de 384 números que captura su significado en este contexto específico.

In [11]:
# Visualizamos el embedding del primer token
import numpy as np

primer_token_embedding = output[0][0].detach().numpy()

print("Embedding del primer token [CLS]:")
print(f"Primeros 10 valores: {primer_token_embedding[:10]}")
print(f"\nEstadísticas del vector:")
print(f"- Media: {primer_token_embedding.mean():.4f}")
print(f"- Desviación estándar: {primer_token_embedding.std():.4f}")
print(f"- Mínimo: {primer_token_embedding.min():.4f}")
print(f"- Máximo: {primer_token_embedding.max():.4f}")

Embedding del primer token [CLS]:
Primeros 10 valores: [-3.366619    0.12981969 -0.08033393 -0.4157191  -0.06599673 -0.34292436
 -0.0917313  -0.20154317 -0.23353307 -0.17304106]

Estadísticas del vector:
- Media: -0.0140
- Desviación estándar: 0.8162
- Mínimo: -3.6123
- Máximo: 4.0823


## Parte 3: Embeddings de Oraciones Completas

Para tareas como búsqueda semántica o clustering, necesitamos representar oraciones completas como un solo vector. Usamos **Sentence-BERT** para esto.

In [12]:
from sentence_transformers import SentenceTransformer

# Cargamos un modelo de embeddings de oraciones
# all-mpnet-base-v2: Uno de los mejores modelos multilingües
modelo_oraciones = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

print("Modelo de sentence embeddings cargado")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo de sentence embeddings cargado


### Sentence-BERT

**Sentence-BERT (SBERT)** es una modificación de BERT optimizada para generar embeddings de oraciones de alta calidad:

- Entrena usando pares de oraciones (similares y diferentes)
- Genera un único vector que representa toda la oración
- Permite comparar oraciones eficientemente (búsqueda semántica)
- Funciona bien en múltiples idiomas, incluyendo español

**Modelo all-mpnet-base-v2:**
- Basado en Microsoft MPNet
- Entrenado con más de 1 billón de pares de oraciones
- 768 dimensiones por oración
- Excelente balance entre calidad y velocidad

In [13]:
# Generamos embedding de una oración
oracion_ejemplo = "La mejor película que vi este año fue increíble"

# encode(): Convierte texto en embedding
vector = modelo_oraciones.encode(oracion_ejemplo)

print(f"Oración: '{oracion_ejemplo}'")
print(f"\nDimensiones del embedding: {vector.shape}")
print(f"Tipo de datos: {type(vector)}")
print(f"\nPrimeros 10 valores del vector:")
print(vector[:10])

Oración: 'La mejor película que vi este año fue increíble'

Dimensiones del embedding: (768,)
Tipo de datos: <class 'numpy.ndarray'>

Primeros 10 valores del vector:
[-0.02195267 -0.02486978 -0.03328351 -0.02514962  0.0163444   0.03398255
 -0.01638375  0.01776548  0.06007986  0.00367057]


### Aplicación: Similitud Semántica

Una aplicación práctica de los embeddings es calcular la similitud entre oraciones.

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# Definimos varias oraciones para comparar
oraciones = [
    "El perro corre por el parque",
    "Un can juega en el espacio verde",
    "El gato duerme en el sillón",
    "Hace mucho calor en verano",
    "La temperatura está muy alta en esta época"
]

# Generamos embeddings para todas las oraciones
embeddings = modelo_oraciones.encode(oraciones)

print(f"Generados {len(embeddings)} embeddings de {embeddings[0].shape[0]} dimensiones\n")

# Calculamos similitud entre la primera oración y las demás
print(f"Oración de referencia: '{oraciones[0]}'\n")
print("Similitud con otras oraciones:")
print("-" * 70)

referencia = embeddings[0].reshape(1, -1)

for i, oracion in enumerate(oraciones[1:], 1):
    similitud = cosine_similarity(referencia, embeddings[i].reshape(1, -1))[0][0]
    print(f"{similitud:.4f} | {oracion}")

Generados 5 embeddings de 768 dimensiones

Oración de referencia: 'El perro corre por el parque'

Similitud con otras oraciones:
----------------------------------------------------------------------
0.4842 | Un can juega en el espacio verde
0.4743 | El gato duerme en el sillón
0.5844 | Hace mucho calor en verano
0.4495 | La temperatura está muy alta en esta época


### Interpretando la Similitud Coseno

La **similitud coseno** mide el ángulo entre dos vectores:

- **1.0**: Vectores idénticos (máxima similitud)
- **0.0**: Vectores perpendiculares (sin relación)
- **-1.0**: Vectores opuestos (mínima similitud)

En la práctica:
- **> 0.8**: Oraciones muy similares (paráfrasis)
- **0.5-0.8**: Similitud moderada (tema relacionado)
- **< 0.5**: Similitud baja (temas diferentes)

En nuestro ejemplo, esperamos que:
- "Un can juega en el espacio verde" tenga alta similitud (es una paráfrasis)
- "El gato duerme en el sillón" tenga similitud media (tema relacionado: mascotas)
- "Hace mucho calor en verano" tenga baja similitud (tema diferente)

## Ejercicio Práctico: Búsqueda Semántica

Vamos a implementar un buscador semántico simple.

In [15]:
# Base de datos de documentos (simulada)
documentos = [
    "El mate es una bebida tradicional argentina que se prepara con yerba",
    "El asado argentino se caracteriza por su cocción lenta sobre brasas",
    "Buenos Aires es la capital de Argentina y su ciudad más poblada",
    "El tango es un género musical nacido en los suburbios de Buenos Aires",
    "La Patagonia argentina es conocida por sus paisajes montañosos y glaciares",
    "El dulce de leche es un producto típico de la repostería rioplatense",
    "El fútbol es el deporte más popular en Argentina",
    "La lengua oficial de Argentina es el español rioplatense"
]

# Generamos embeddings de todos los documentos
embeddings_documentos = modelo_oraciones.encode(documentos)
print(f"Base de datos indexada: {len(documentos)} documentos\n")

# Función de búsqueda
def buscar(consulta, top_k=3):
    """
    Busca los documentos más similares a la consulta.

    Parámetros:
    - consulta: Texto de búsqueda
    - top_k: Cantidad de resultados a devolver
    """
    # Generamos embedding de la consulta
    embedding_consulta = modelo_oraciones.encode([consulta])

    # Calculamos similitudes
    similitudes = cosine_similarity(embedding_consulta, embeddings_documentos)[0]

    # Obtenemos los índices de los top_k más similares
    # argsort(): Ordena y devuelve índices
    # [::-1]: Invertimos para orden descendente
    # [:top_k]: Tomamos los primeros k
    indices_top = similitudes.argsort()[::-1][:top_k]

    print(f"Consulta: '{consulta}'\n")
    print("Resultados:")
    print("=" * 80)
    for rank, idx in enumerate(indices_top, 1):
        print(f"\n{rank}. Similitud: {similitudes[idx]:.4f}")
        print(f"   {documentos[idx]}")

# Probamos varias consultas
buscar("¿Qué comidas típicas hay?", top_k=3)

Base de datos indexada: 8 documentos

Consulta: '¿Qué comidas típicas hay?'

Resultados:

1. Similitud: 0.5316
   El asado argentino se caracteriza por su cocción lenta sobre brasas

2. Similitud: 0.5267
   El dulce de leche es un producto típico de la repostería rioplatense

3. Similitud: 0.5202
   El mate es una bebida tradicional argentina que se prepara con yerba


In [16]:
print("\n" + "="*80 + "\n")
buscar("Información sobre música y baile", top_k=3)



Consulta: 'Información sobre música y baile'

Resultados:

1. Similitud: 0.5089
   El tango es un género musical nacido en los suburbios de Buenos Aires

2. Similitud: 0.3460
   El dulce de leche es un producto típico de la repostería rioplatense

3. Similitud: 0.3362
   El asado argentino se caracteriza por su cocción lenta sobre brasas


In [17]:
print("\n" + "="*80 + "\n")
buscar("Lugares para visitar", top_k=3)



Consulta: 'Lugares para visitar'

Resultados:

1. Similitud: 0.4803
   Buenos Aires es la capital de Argentina y su ciudad más poblada

2. Similitud: 0.4789
   El dulce de leche es un producto típico de la repostería rioplatense

3. Similitud: 0.4724
   El asado argentino se caracteriza por su cocción lenta sobre brasas


### Ventajas de la Búsqueda Semántica

A diferencia de la búsqueda por palabras clave tradicional:

1. **Comprende intención**: "comidas típicas" encuentra "mate", "asado", "dulce de leche" aunque no compartan palabras exactas
2. **Maneja sinónimos**: "música y baile" encuentra "tango" sin necesidad de diccionarios de sinónimos
3. **Robusta a variaciones**: Funciona con diferentes formas de preguntar lo mismo
4. **Multilingüe**: Puede buscar en un idioma y encontrar resultados en otro (según el modelo)

Esta tecnología se usa en:
- Motores de búsqueda modernos
- Sistemas de recomendación
- Chatbots y asistentes virtuales
- Detección de duplicados y plagio

## Experimentación Libre

Probá tu propio buscador:

In [ ]:
# Agregá tus propios documentos
mis_documentos = [
    "Tu documento 1",
    "Tu documento 2",
    "Tu documento 3",
    # Agrega más...
]

# Indexá
mis_embeddings = modelo_oraciones.encode(mis_documentos)

# Buscá
mi_consulta = "Tu consulta aquí"
embedding_consulta = modelo_oraciones.encode([mi_consulta])
similitudes = cosine_similarity(embedding_consulta, mis_embeddings)[0]

# Mostrá resultados
for i, doc in enumerate(mis_documentos):
    print(f"{similitudes[i]:.4f} | {doc}")

## Resumen de Conceptos Clave

### Tokenización

1. **Tokens**: Unidades mínimas que procesan los modelos (palabras, subpalabras, caracteres)
2. **Algoritmos**: BPE, WordPiece, SentencePiece dividen texto en subpalabras
3. **Diferencias entre modelos**: Cada modelo tiene su propia estrategia de tokenización
4. **Impacto**: Afecta eficiencia, manejo de idiomas, y representación de palabras raras

### Embeddings

1. **Representación vectorial**: Texto convertido en números (vectores)
2. **Embeddings contextualizados**: El vector cambia según el contexto (BERT, DeBERTa)
3. **Embeddings de oraciones**: Un vector para toda la oración (Sentence-BERT)
4. **Similitud semántica**: Vectores similares representan significados similares

### Aplicaciones Prácticas

1. **Búsqueda semántica**: Encontrar documentos por significado, no por palabras
2. **Clustering**: Agrupar textos similares
3. **Clasificación**: Categorizar textos basándose en embeddings
4. **Sistemas de recomendación**: Sugerir contenido similar
5. **Detección de duplicados**: Identificar textos equivalentes

## Guía de Preguntas y Respuestas

### Preguntas de Comprensión

**1. ¿Qué es un token y por qué los modelos usan subpalabras en lugar de palabras completas?**

<details>
<summary>Ver respuesta</summary>

Un token es la unidad mínima de texto que un modelo procesa. Los modelos usan subpalabras por varias razones:

1. **Vocabulario manejable**: En lugar de millones de palabras posibles, usan 30,000-50,000 subpalabras
2. **Palabras raras**: Pueden representar palabras que nunca vieron combinando subpalabras conocidas
3. **Eficiencia morfológica**: Capturan raíces y afijos ("jugador", "jugando", "jugar" comparten "jug")
4. **Multilingüismo**: Un vocabulario puede cubrir múltiples idiomas

Ejemplo: "jugando" podría tokenizarse como ["jug", "ando"], permitiendo que el modelo reconozca la raíz "jug" en otras palabras.
</details>

---

**2. ¿Qué diferencia hay entre embeddings estáticos y embeddings contextualizados?**

<details>
<summary>Ver respuesta</summary>

**Embeddings estáticos** (Word2Vec, GloVe):
- Cada palabra tiene un único vector fijo
- "banco" siempre tiene el mismo vector, independientemente del contexto
- Problema: No distingue "banco" (institución financiera) de "banco" (asiento)

**Embeddings contextualizados** (BERT, DeBERTa, GPT):
- El vector cambia según el contexto de la oración
- "banco" en "banco de datos" tiene un vector diferente que en "sentarse en el banco"
- Ventaja: Captura el significado específico en cada uso

Los modelos modernos usan embeddings contextualizados porque son mucho más precisos para entender el significado real de las palabras.
</details>

---

**3. ¿Qué representa el shape [1, 5, 384] en un tensor de embeddings?**

<details>
<summary>Ver respuesta</summary>

- **1**: Batch size (número de ejemplos procesados simultáneamente)
- **5**: Longitud de la secuencia (número de tokens en la oración, incluyendo tokens especiales como [CLS] y [SEP])
- **384**: Hidden size (dimensiones del embedding, cada token se representa con 384 números)

Ejemplo: Si procesamos "Hola mundo" con DeBERTa v3-xsmall:
- Se tokeniza como [CLS] Hola mundo [SEP] = 4 tokens
- Cada token se convierte en un vector de 384 dimensiones
- Shape final: [1, 4, 384]
</details>

---

**4. ¿Qué es la similitud coseno y cómo se interpreta?**

<details>
<summary>Ver respuesta</summary>

La **similitud coseno** mide el ángulo entre dos vectores en un espacio multidimensional. Sus valores van de -1 a 1:

- **1.0**: Vectores idénticos (máxima similitud)
- **0.0**: Vectores perpendiculares (sin relación)
- **-1.0**: Vectores opuestos (mínima similitud)

En NLP, interpretamos:
- **> 0.8**: Oraciones muy similares (paráfrasis, mismo tema)
- **0.5-0.8**: Similitud moderada (temas relacionados)
- **< 0.5**: Similitud baja (temas diferentes)

Ventaja sobre la distancia euclidiana: No se afecta por la magnitud del vector, solo por la dirección (orientación en el espacio).
</details>

---

**5. ¿Qué es Sentence-BERT y en qué se diferencia de BERT?**

<details>
<summary>Ver respuesta</summary>

**BERT**:
- Genera un embedding por cada token
- Para obtener un embedding de oración, hay que promediar o tomar el token [CLS]
- No fue entrenado específicamente para comparar oraciones

**Sentence-BERT (SBERT)**:
- Modificación de BERT entrenada con pares de oraciones
- Genera un único vector de alta calidad para toda la oración
- Optimizado para tareas de similitud y búsqueda semántica
- Mucho más eficiente para comparar oraciones (miles de veces más rápido)

Usos principales de SBERT:
- Búsqueda semántica
- Clustering de documentos
- Detección de paráfrasis
- Sistemas de recomendación
</details>

---

### Preguntas de Aplicación

**6. Si observás que BERT uncased genera más tokens que GPT-2 para el mismo texto español, ¿qué podría explicar esta diferencia?**

<details>
<summary>Ver respuesta</summary>

Posibles explicaciones:

1. **Vocabulario de entrenamiento**: GPT-2 fue entrenado con más datos en español, por lo que su vocabulario incluye más palabras españolas completas

2. **Algoritmo de tokenización**: Diferentes algoritmos (WordPiece vs BPE) fragmentan texto de manera distinta

3. **Manejo de mayúsculas**: BERT uncased convierte todo a minúsculas, lo que podría no ser óptimo para preservar información

4. **Tamaño del vocabulario**: Vocabularios más grandes tienden a generar menos tokens

Para texto español, modelos específicamente entrenados en español (como BETO) serían más eficientes.
</details>

---

**7. ¿Cómo modificarías el código para implementar un clasificador de temas basado en embeddings?**

<details>
<summary>Ver respuesta</summary>

Estrategia:

1. **Crear embeddings de referencia por categoría**:
```python
categorias = {
    "deportes": modelo_oraciones.encode(["fútbol", "tenis", "competencia"]).mean(axis=0),
    "tecnología": modelo_oraciones.encode(["computadora", "software", "programación"]).mean(axis=0),
    "cocina": modelo_oraciones.encode(["receta", "ingredientes", "cocinar"]).mean(axis=0)
}
```

2. **Clasificar nuevo texto**:
```python
def clasificar(texto):
    embedding_texto = modelo_oraciones.encode([texto])
    similitudes = {}
    for categoria, embedding_cat in categorias.items():
        sim = cosine_similarity(embedding_texto, embedding_cat.reshape(1, -1))[0][0]
        similitudes[categoria] = sim
    return max(similitudes, key=similitudes.get)
```

Alternativamente, podés entrenar un clasificador supervisado (SVM, red neuronal) usando los embeddings como features.
</details>

---

**8. Tu buscador semántico devuelve resultados poco relevantes. ¿Qué estrategias podrías usar para mejorarlo?**

<details>
<summary>Ver respuesta</summary>

Estrategias de mejora:

1. **Usar un modelo mejor**: Probar modelos más grandes o específicos del dominio
   - Para español: `hiiamsid/sentence_similarity_spanish_es`
   - Multilingüe mejorado: `sentence-transformers/paraphrase-multilingual-mpnet-base-v2`

2. **Pre-procesamiento**:
   - Limpiar texto (remover stopwords menos importantes)
   - Normalizar (minúsculas, acentos)

3. **Re-ranking**: Usar un modelo de dos etapas
   - Primera pasada: Recuperar top 20 con embeddings (rápido)
   - Segunda pasada: Re-ordenar con modelo más sofisticado (lento pero preciso)

4. **Ajustar umbral**: Establecer un mínimo de similitud para filtrar resultados irrelevantes

5. **Fine-tuning**: Entrenar el modelo con ejemplos de tu dominio específico

6. **Hybrid search**: Combinar búsqueda semántica con búsqueda por palabras clave
</details>

---

**9. ¿Cómo calcularías el embedding promedio de un documento largo que excede el límite de tokens del modelo?**

<details>
<summary>Ver respuesta</summary>

Estrategias:

1. **Chunking con promedio**:
```python
def embedding_documento_largo(texto, modelo, max_length=512):
    # Dividir en chunks
    palabras = texto.split()
    chunks = [' '.join(palabras[i:i+max_length])
              for i in range(0, len(palabras), max_length)]
    
    # Obtener embeddings de cada chunk
    embeddings = modelo.encode(chunks)
    
    # Promediar
    return embeddings.mean(axis=0)
```

2. **Sliding window**: Chunks con solapamiento para no perder contexto

3. **Usar solo el inicio**: Si el documento sigue una estructura estándar (resumen al principio)

4. **Summarización primero**: Resumir el documento y luego obtener embedding

5. **Modelos de contexto largo**: Usar modelos con mayor límite (Longformer, BigBird)
</details>

---

**10. Implementá una función que agrupe automáticamente noticias similares usando clustering con embeddings.**

<details>
<summary>Ver respuesta</summary>

```python
from sklearn.cluster import KMeans
import numpy as np

def agrupar_noticias(noticias, n_clusters=3):
    """
    Agrupa noticias similares usando K-means sobre embeddings.
    
    Parámetros:
    - noticias: Lista de textos de noticias
    - n_clusters: Número de grupos a formar
    """
    # Generar embeddings
    embeddings = modelo_oraciones.encode(noticias)
    
    # Clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    
    # Organizar resultados
    grupos = {i: [] for i in range(n_clusters)}
    for noticia, cluster in zip(noticias, clusters):
        grupos[cluster].append(noticia)
    
    # Mostrar
    for i, noticias_cluster in grupos.items():
        print(f"\nGrupo {i+1}:")
        for noticia in noticias_cluster:
            print(f"  - {noticia[:60]}...")
    
    return grupos

# Ejemplo de uso:
noticias = [
    "El gobierno anuncia nuevas medidas económicas",
    "Récord de asistencia en el partido de fútbol",
    "La inflación alcanza un nuevo máximo histórico",
    "El equipo nacional clasifica al mundial",
    "Aumentan las tasas de interés bancarias"
]

agrupar_noticias(noticias, n_clusters=2)
```

Esto agrupará automáticamente noticias de economía vs deportes.
</details>

---

### Preguntas de Reflexión

**11. ¿Por qué los embeddings contextualizados son superiores a los estáticos para la mayoría de las tareas de NLP modernas?**

<details>
<summary>Ver respuesta</summary>

Los embeddings contextualizados son superiores porque:

1. **Desambiguación**: Resuelven la polisemia (palabras con múltiples significados)
   - "banco" financiero vs "banco" de plaza
   - "vino" (bebida) vs "vino" (verbo venir)

2. **Capturan sintaxis**: Entienden relaciones gramaticales y dependencias

3. **Adaptación al contexto**: El mismo concepto puede tener matices diferentes según el contexto

4. **Mayor precisión**: En tareas como QA, NER, sentiment analysis, superan consistentemente a embeddings estáticos

5. **Transfer learning**: Pre-entrenados en grandes corpus, se adaptan bien a dominios específicos

Sin embargo, embeddings estáticos aún tienen ventajas:
- Más rápidos y livianos
- Útiles cuando el contexto no importa (ej: búsqueda de palabras clave)
- Interpretativos (puedes analizar el espacio de embeddings)
</details>

---

**12. ¿Qué consideraciones éticas hay al usar embeddings para sistemas de recomendación o búsqueda?**

<details>
<summary>Ver respuesta</summary>

Consideraciones importantes:

1. **Sesgos en los datos**: Los embeddings reflejan sesgos del corpus de entrenamiento
   - Género: "doctor" podría asociarse más con "hombre" que "mujer"
   - Raza, nacionalidad, religión: Pueden contener estereotipos

2. **Burbujas de filtro**: Recomendar solo contenido similar puede limitar la exposición a ideas diversas

3. **Privacidad**: Los embeddings pueden revelar información sensible sobre preferencias del usuario

4. **Representación cultural**: Modelos entrenados mayormente en inglés pueden ser menos efectivos para otras culturas

5. **Explicabilidad**: Es difícil explicar por qué un embedding considera dos cosas similares

Mitigaciones:
- Auditar embeddings para detectar sesgos
- Usar datos de entrenamiento diversos y balanceados
- Implementar diversidad en recomendaciones
- Ser transparente sobre limitaciones del sistema
- Permitir control del usuario sobre las recomendaciones
</details>

---

### Desafíos Adicionales

**Desafío 1**: Implementá un sistema que detecte noticias duplicadas o muy similares en un conjunto de datos, usando un umbral de similitud configurable.

**Desafío 2**: Creá un visualizador de embeddings usando PCA o t-SNE para reducir las dimensiones a 2D y graficar con matplotlib.

**Desafío 3**: Construí un sistema de recomendación de artículos: dado un artículo que te gustó, encuentra los 5 más similares en una colección.

---

## Referencias y Recursos Adicionales

- [Sentence-BERT Paper](https://arxiv.org/abs/1908.10084)
- [BERT Paper](https://arxiv.org/abs/1810.04805)
- [DeBERTa Paper](https://arxiv.org/abs/2006.03654)
- [Hugging Face Tokenizers](https://huggingface.co/docs/tokenizers/index)
- [Sentence-Transformers Documentation](https://www.sbert.net/)